# Analisi dei dati raccolti

## Setup

In [1]:
import pandas as pd

In [2]:
def print_data(series: pd.Series) -> None:
    print(series.value_counts())
    print()
    print(f"Massimo: {series.max()}\nMinimo: {series.min()}\nMedia: {series.mean()}\nVarianza: {series.var()}")

## Ascon128v12

In [3]:
path = "../ascon128v12/data/{}.csv"

### Armv7m

In [4]:
with open(path.format("armv7m"), "r") as f:
    a128_armv7m = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [5]:
# Cifratura
data = a128_armv7m["0B-E"]

print_data(data)

0B-E
47    999
46      1
Name: count, dtype: int64

Massimo: 47
Minimo: 46
Media: 46.999
Varianza: 0.0010000000000000002


In [6]:
# Decifratura
data = a128_armv7m["0B-D"]

print_data(data)

0B-D
94    799
95    200
93      1
Name: count, dtype: int64

Massimo: 95
Minimo: 93
Media: 94.199
Varianza: 0.16156056056056062


#### PT 1 byte

In [7]:
# Cifratura
data = a128_armv7m["1B-E"]

print_data(data)

1B-E
60    999
61      1
Name: count, dtype: int64

Massimo: 61
Minimo: 60
Media: 60.001
Varianza: 0.0010000000000000002


In [8]:
# Decifratura
data = a128_armv7m["1B-D"]

print_data(data)

1B-D
125    999
126      1
Name: count, dtype: int64

Massimo: 126
Minimo: 125
Media: 125.001
Varianza: 0.0010000000000000002


#### PT 8 byte

In [9]:
# Cifratura
data = a128_armv7m["8B-E"]

print_data(data)

8B-E
80    800
81    200
Name: count, dtype: int64

Massimo: 81
Minimo: 80
Media: 80.2
Varianza: 0.1601601601601601


In [10]:
# Decifratura
data = a128_armv7m["8B-D"]

print_data(data)

8B-D
162    799
163    201
Name: count, dtype: int64

Massimo: 163
Minimo: 162
Media: 162.201
Varianza: 0.16075975975975976


#### PT 16 byte

In [11]:
# Cifratura
data = a128_armv7m["16B-E"]

print_data(data)

16B-E
102    997
103      3
Name: count, dtype: int64

Massimo: 103
Minimo: 102
Media: 102.003
Varianza: 0.002993993993993996


In [12]:
# Decifratura
data = a128_armv7m["16B-D"]

print_data(data)

16B-D
207    800
208    200
Name: count, dtype: int64

Massimo: 208
Minimo: 207
Media: 207.2
Varianza: 0.1601601601601602


#### PT 24 byte

In [13]:
# Cifratura
data = a128_armv7m["24B-E"]

print_data(data)

24B-E
125    799
126    201
Name: count, dtype: int64

Massimo: 126
Minimo: 125
Media: 125.201
Varianza: 0.16075975975975976


In [14]:
# Decifratura
data = a128_armv7m["24B-D"]

print_data(data)

24B-D
252    799
253    201
Name: count, dtype: int64

Massimo: 253
Minimo: 252
Media: 252.201
Varianza: 0.16075975975975976


#### PT 32 byte

In [15]:
# Cifratura
data = a128_armv7m["32B-E"]

print_data(data)

32B-E
147    799
148    201
Name: count, dtype: int64

Massimo: 148
Minimo: 147
Media: 147.201
Varianza: 0.16075975975975976


In [16]:
# Decifratura
data = a128_armv7m["32B-D"]

print_data(data)

32B-D
297    800
298    200
Name: count, dtype: int64

Massimo: 298
Minimo: 297
Media: 297.2
Varianza: 0.1601601601601602


### Armv7m_lowsize

In [17]:
with open(path.format("armv7m_lowsize"), "r") as f:
    a128_lowsize = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [18]:
# Cifratura
data = a128_lowsize["0B-E"]

print_data(data)

0B-E
44    957
45     43
Name: count, dtype: int64

Massimo: 45
Minimo: 44
Media: 44.043
Varianza: 0.0411921921921922


In [19]:
# Decifratura
data = a128_lowsize["0B-D"]

print_data(data)

0B-D
86    921
88     79
Name: count, dtype: int64

Massimo: 88
Minimo: 86
Media: 86.158
Varianza: 0.29132732732732725


#### PT 1 byte

In [20]:
# Cifratura
data = a128_lowsize["1B-E"]

print_data(data)

1B-E
56    919
57     81
Name: count, dtype: int64

Massimo: 57
Minimo: 56
Media: 56.081
Varianza: 0.07451351351351351


In [21]:
# Decifratura
data = a128_lowsize["1B-D"]

print_data(data)

1B-D
110    880
111    120
Name: count, dtype: int64

Massimo: 111
Minimo: 110
Media: 110.12
Varianza: 0.1057057057057057


#### PT 8 byte

In [22]:
# Cifratura
data = a128_lowsize["8B-E"]

print_data(data)

8B-E
75    920
76     80
Name: count, dtype: int64

Massimo: 76
Minimo: 75
Media: 75.08
Varianza: 0.0736736736736737


In [23]:
# Decifratura
data = a128_lowsize["8B-D"]

print_data(data)

8B-D
148    878
149    122
Name: count, dtype: int64

Massimo: 149
Minimo: 148
Media: 148.122
Varianza: 0.10722322322322324


#### PT 16 byte

In [24]:
# Cifratura
data = a128_lowsize["16B-E"]

print_data(data)

16B-E
93    918
95     80
94      2
Name: count, dtype: int64

Massimo: 95
Minimo: 93
Media: 93.162
Varianza: 0.29605205205205204


In [25]:
# Decifratura
data = a128_lowsize["16B-D"]

print_data(data)

16B-D
186    840
187    160
Name: count, dtype: int64

Massimo: 187
Minimo: 186
Media: 186.16
Varianza: 0.13453453453453457


#### PT 24 byte

In [26]:
# Cifratura
data = a128_lowsize["24B-E"]

print_data(data)

24B-E
112    880
113    119
114      1
Name: count, dtype: int64

Massimo: 114
Minimo: 112
Media: 112.121
Varianza: 0.10846746746746745


In [27]:
# Decifratura
data = a128_lowsize["24B-D"]

print_data(data)

24B-D
224    799
226    199
225      2
Name: count, dtype: int64

Massimo: 226
Minimo: 224
Media: 224.4
Varianza: 0.6386386386386385


#### PT 32 byte

In [28]:
# Cifratura
data = a128_lowsize["32B-E"]

print_data(data)

32B-E
131    881
132    119
Name: count, dtype: int64

Massimo: 132
Minimo: 131
Media: 131.119
Varianza: 0.10494394394394395


In [29]:
# Decifratura
data = a128_lowsize["32B-D"]

print_data(data)

32B-D
263    759
264    240
262      1
Name: count, dtype: int64

Massimo: 264
Minimo: 262
Media: 263.239
Varianza: 0.18406306306306303


### Armv7m_small

In [30]:
with open(path.format("armv7m_small"), "r") as f:
    a128_small = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [31]:
# Cifratura
data = a128_small["0B-E"]

print_data(data)

0B-E
38    959
39     40
37      1
Name: count, dtype: int64

Massimo: 39
Minimo: 37
Media: 38.039
Varianza: 0.03951851851851851


In [32]:
# Decifratura
data = a128_small["0B-D"]

print_data(data)

0B-D
77    918
78     79
76      3
Name: count, dtype: int64

Massimo: 78
Minimo: 76
Media: 77.076
Varianza: 0.0763003003003003


#### PT 1 byte

In [33]:
# Cifratura
data = a128_small["1B-E"]

print_data(data)

1B-E
49    919
50     80
51      1
Name: count, dtype: int64

Massimo: 51
Minimo: 49
Media: 49.082
Varianza: 0.07735335335335337


In [34]:
# Decifratura
data = a128_small["1B-D"]

print_data(data)

1B-D
102    919
103     80
101      1
Name: count, dtype: int64

Massimo: 103
Minimo: 101
Media: 102.079
Varianza: 0.07483383383383382


#### PT 8 byte

In [35]:
# Cifratura
data = a128_small["8B-E"]

print_data(data)

8B-E
66    920
67     80
Name: count, dtype: int64

Massimo: 67
Minimo: 66
Media: 66.08
Varianza: 0.0736736736736737


In [36]:
# Decifratura
data = a128_small["8B-D"]

print_data(data)

8B-D
133    879
134    120
132      1
Name: count, dtype: int64

Massimo: 134
Minimo: 132
Media: 133.119
Varianza: 0.10694594594594596


#### PT 16 byte

In [37]:
# Cifratura
data = a128_small["16B-E"]

print_data(data)

16B-E
84    879
85     81
86     40
Name: count, dtype: int64

Massimo: 86
Minimo: 84
Media: 84.161
Varianza: 0.21529429429429428


In [38]:
# Decifratura
data = a128_small["16B-D"]

print_data(data)

16B-D
170    841
171    159
Name: count, dtype: int64

Massimo: 171
Minimo: 170
Media: 170.159
Varianza: 0.1338528528528529


#### PT 24 byte

In [39]:
# Cifratura
data = a128_small["24B-E"]

print_data(data)

24B-E
102    879
104    120
103      1
Name: count, dtype: int64

Massimo: 104
Minimo: 102
Media: 102.241
Varianza: 0.42334234234234236


In [40]:
# Decifratura
data = a128_small["24B-D"]

print_data(data)

24B-D
207    800
208    200
Name: count, dtype: int64

Massimo: 208
Minimo: 207
Media: 207.2
Varianza: 0.1601601601601602


#### PT 32 byte

In [41]:
# Cifratura
data = a128_small["32B-E"]

print_data(data)

32B-E
121    880
122    120
Name: count, dtype: int64

Massimo: 122
Minimo: 121
Media: 121.12
Varianza: 0.1057057057057057


In [42]:
# Decifratura
data = a128_small["32B-D"]

print_data(data)

32B-D
244    758
246    241
245      1
Name: count, dtype: int64

Massimo: 246
Minimo: 244
Media: 244.483
Varianza: 0.7324434434434437


### Bi32_armv7m

In [43]:
with open(path.format("bi32_armv7m"), "r") as f:
    a128_bi32 = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [44]:
# Cifratura
data = a128_bi32["0B-E"]

print_data(data)

0B-E
39    959
41     40
40      1
Name: count, dtype: int64

Massimo: 41
Minimo: 39
Media: 39.081
Varianza: 0.15459359359359356


In [45]:
# Decifratura
data = a128_bi32["0B-D"]

print_data(data)

0B-D
79    921
80     79
Name: count, dtype: int64

Massimo: 80
Minimo: 79
Media: 79.079
Varianza: 0.07283183183183181


#### PT 1 byte

In [46]:
# Cifratura
data = a128_bi32["1B-E"]

print_data(data)

1B-E
53    959
54     41
Name: count, dtype: int64

Massimo: 54
Minimo: 53
Media: 53.041
Varianza: 0.039358358358358346


In [47]:
# Cifratura
data = a128_bi32["1B-D"]

print_data(data)

1B-D
107    878
109    120
108      2
Name: count, dtype: int64

Massimo: 109
Minimo: 107
Media: 107.242
Varianza: 0.42385985985986


#### PT 8 byte

In [48]:
# Cifratura
data = a128_bi32["8B-E"]

print_data(data)

8B-E
68    918
69     80
67      2
Name: count, dtype: int64

Massimo: 69
Minimo: 67
Media: 68.078
Varianza: 0.07599199199199196


In [49]:
# Decifratura
data = a128_bi32["8B-D"]

print_data(data)

8B-D
137    877
139    120
138      3
Name: count, dtype: int64

Massimo: 139
Minimo: 137
Media: 137.243
Varianza: 0.4243753753753754


#### PT 16 byte

In [50]:
# Cifratura
data = a128_bi32["16B-E"]

print_data(data)

16B-E
88    918
89     81
87      1
Name: count, dtype: int64

Massimo: 89
Minimo: 87
Media: 88.08
Varianza: 0.0756756756756757


In [51]:
# Decifratura
data = a128_bi32["16B-D"]

print_data(data)

16B-D
178    838
179    162
Name: count, dtype: int64

Massimo: 179
Minimo: 178
Media: 178.162
Varianza: 0.13589189189189188


#### PT 24 byte

In [52]:
# Cifratura
data = a128_bi32["24B-E"]

print_data(data)

24B-E
107    879
108     79
109     42
Name: count, dtype: int64

Massimo: 109
Minimo: 107
Media: 107.163
Varianza: 0.22065165165165165


In [53]:
# Decifratura
data = a128_bi32["24B-D"]

print_data(data)

24B-D
218    798
220    200
219      2
Name: count, dtype: int64

Massimo: 220
Minimo: 218
Media: 218.402
Varianza: 0.641037037037037


#### PT 32 byte

In [54]:
# Cifratura
data = a128_bi32["32B-E"]

print_data(data)

32B-E
127    879
128    120
126      1
Name: count, dtype: int64

Massimo: 128
Minimo: 126
Media: 127.119
Varianza: 0.10694594594594595


In [55]:
# Decifratura
data = a128_bi32["32B-D"]

print_data(data)

32B-D
259    758
260    240
261      2
Name: count, dtype: int64

Massimo: 261
Minimo: 259
Media: 259.244
Varianza: 0.1886526526526527


### Ref

In [56]:
with open(path.format("ref"), "r") as f:
    a128_ref = pd.read_csv(f, delimiter=";")

#### PT vuoto

In [57]:
# Cifratura
data = a128_ref["0B-E"]

print_data(data)

0B-E
147    880
148    120
Name: count, dtype: int64

Massimo: 148
Minimo: 147
Media: 147.12
Varianza: 0.1057057057057057


In [58]:
# Decifratura
data = a128_ref["0B-D"]

print_data(data)

0B-D
301    678
303    240
302     82
Name: count, dtype: int64

Massimo: 303
Minimo: 301
Media: 301.562
Varianza: 0.7268828828828827


#### PT 1 byte

In [59]:
# Cifratura
data = a128_ref["1B-E"]

print_data(data)

1B-E
178    839
179    160
177      1
Name: count, dtype: int64

Massimo: 179
Minimo: 177
Media: 178.159
Varianza: 0.1358548548548549


In [60]:
# Decifratura
data = a128_ref["1B-D"]

print_data(data)

1B-D
364    598
366    319
365     82
367      1
Name: count, dtype: int64

Massimo: 367
Minimo: 364
Media: 364.723
Varianza: 0.8451161161161163


#### PT 8 byte

In [61]:
# Cifratura
data = a128_ref["8B-E"]

print_data(data)

8B-E
247    762
248    238
Name: count, dtype: int64

Massimo: 248
Minimo: 247
Media: 247.238
Varianza: 0.1815375375375376


In [62]:
# Decifratura
data = a128_ref["8B-D"]

print_data(data)

8B-D
504    519
503    481
Name: count, dtype: int64

Massimo: 504
Minimo: 503
Media: 503.519
Varianza: 0.2498888888888889


#### PT 16 byte

In [63]:
# Cifratura
data = a128_ref["16B-E"]

print_data(data)

16B-E
318    637
319    362
320      1
Name: count, dtype: int64

Massimo: 320
Minimo: 318
Media: 318.364
Varianza: 0.2337377377377377


In [64]:
# Decifratura
data = a128_ref["16B-D"]

print_data(data)

16B-D
647    681
646    319
Name: count, dtype: int64

Massimo: 647
Minimo: 646
Media: 646.681
Varianza: 0.21745645645645642


#### PT 24 byte

In [65]:
# Cifratura
data = a128_ref["24B-E"]

print_data(data)

24B-E
389    557
390    401
391     42
Name: count, dtype: int64

Massimo: 391
Minimo: 389
Media: 389.485
Varianza: 0.33410910910910896


In [66]:
# Decifratura
data = a128_ref["24B-D"]

print_data(data)

24B-D
790    678
788    200
789    121
791      1
Name: count, dtype: int64

Massimo: 791
Minimo: 788
Media: 789.48
Varianza: 0.652252252252252


#### PT 32 byte

In [67]:
# Cifratura
data = a128_ref["32B-E"]

print_data(data)

32B-E
460    521
461    437
462     41
463      1
Name: count, dtype: int64

Massimo: 463
Minimo: 460
Media: 460.522
Varianza: 0.3378538538538538


In [68]:
# Decifratura
data = a128_ref["32B-D"]

print_data(data)

32B-D
932    958
931     40
933      1
934      1
Name: count, dtype: int64

Massimo: 934
Minimo: 931
Media: 931.963
Varianza: 0.04367467467467465
